##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Введение в автоэнкодеры

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/autoencoder">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/autoencoder.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/autoencoder.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/autoencoder.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве представлены три примера автоэнкодеров: основной, шумоподавление изображения и обнаружение аномалий.

Автоэнкодер - это особый тип нейронной сети, которая обучается копировать свои входные данные в выходные данные. Например, учитывая изображение рукописной цифры, автокодер сначала кодирует изображение в скрытое представление меньшей размерности, а затем декодирует скрытое представление обратно в изображение. Автоэнкодер учится сжимать данные, сводя к минимуму ошибку восстановления.

Чтобы узнать больше об автоэнкодерах, прочитайте главу 14 из [Deep Learning](https://www.deeplearningbook.org/) Яна Гудфеллоу, Йошуа Бенжио и Аарона Курвилля.

## Имопорт TensorFlow и библиотек

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

## Загрузка датасета
Для начала вы обучите базовый автоэнкодер, используя набор данных Fashion MNIST. Каждое изображение в этом наборе данных имеет размер 28x28 пикселей.

In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)

## Первый пример: базовый автоэнкодер
![Основные результаты автоэнкодера](images/intro_autoencoder_result.png)

Определите автоэнкодер с двумя полносвязными слоями: `кодировщик`, который сжимает изображения в 64-мерный скрытый вектор, и `декодировщик`, который восстанавливает исходное изображение из скрытого пространства.

Чтобы определить вашу модель, используйте [Keras Model Subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models).


In [ ]:
latent_dim = 64 

class Autoencoder(Model):
  def __init__(self, encoding_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28, 28))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim) 

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

Обучите модель, используя `x_train` как в качестве входных признаков, так и в качестве входных маркеров. `Энкодер` научится сжимать набор данных из 784 измерений в скрытое пространство, а `декодер` научится восстанавливать исходные изображения.

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

Теперь, когда модель обучена, давайте протестируем ее путем кодирования и декодирования изображений из тестового набора.

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

## Второй пример: уменьшение шума на изображении

![Результаты шумоподавления изображения](images/image_denoise_fmnist_results.png)

Автоэнкодер также можно обучить удалению шума с изображений. В следующем разделе вы создадите зашумленную версию набора данных Fashion MNIST, применив случайный шум к каждому изображению. Затем вы обучите автоэнкодер, используя зашумленное изображение в качестве входных данных и исходное изображение в качестве цели.

Давайте повторно импортируем набор данных, чтобы убрать изменения, сделанные ранее.

In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape)

Добавление случайного шума к изображениям

In [ ]:
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape) 
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape) 

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

Посмотрите зашумленные изображения.

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
plt.show()

### Определениее сверточного автоенкодера

В этом примере вы обучите сверточный автоэнкодер, используя слои [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) в `encoder` и [Conv2DTranspose](https : //www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose) в `decoder`.

In [ ]:
class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 1)), 
      layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2)])
    
    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='same')])
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

Давайте взглянем на краткое описание енкодера. Обратите внимание, как изображения уменьшаются с 28x28 до 7x7.

In [ ]:
autoencoder.encoder.summary()

Декодер увеличивает разрешение изображения с 7x7 до 28x28.

In [ ]:
autoencoder.decoder.summary()

Просмотрим зашумленные изображения и изображений с шумоподавлением, созданные автокодером.

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()

## Третий пример: обнаружение аномалии

## Обзор


В этом примере вы обучите автоэнкодер обнаруживать аномалии в [наборе данных ECG5000](http://www.timeseriesclassification.com/description.php?Dataset=ECG5000). Этот набор данных содержит 5000 [электрокардиограмм](https://en.wikipedia.org/wiki/Electrocardiography), каждая из которых содержит 140 точек данных. Вы будете использовать упрощенную версию датасета, где каждый пример помечен как `0`(соответствует ненормальному ритму) или `1`(соответствует нормальному ритму). Вам нужно идентифицировать аномальные ритмы.

Примечание. Это размеченный набор данных, поэтому вы можете назвать его `обучение с учителем`. Цель этого примера - проиллюстрировать концепции обнаружения аномалий, которые вы можете применить к большим наборам данных, где у вас нет размеченных меток(например, если у вас было несколько тысяч нормальных ритмов и только небольшое количество аномальных ритмов).

Как вы обнаружите аномалии с помощью автокодировщика? Напомним, что автоэнкодер обучен минимизировать ошибку восстановления. Вы обучите автоэнкодер только нормальным ритмам, а затем воспользуетесь им для восстановления всех данных. Наша гипотеза состоит в том, что аномальные ритмы будут иметь более высокую ошибку восстановления. Таким образом вы классифицируете ритм как аномалию, если ошибка восстановления превышает фиксированный порог.

### Загрузка ECG датасета

Набор данных, который вы будете использовать, основан на одном из [timeseriesclassification.com](http://www.timeseriesclassification.com/description.php?Dataset=ECG5000).


In [ ]:
# Загружаем датасет
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

In [ ]:
# Последний элемент строки содержит метку
labels = raw_data[:, -1]

# Остальные данные это данные электрокардиограмм
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

Нормализуем данные до `[0,1]`.

In [ ]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

Вы будете обучать автоэнкодер, используя только нормальные ритмы, которые в этом наборе данных обозначены как `1`. Отделите нормальные ритмы от ненормальных.

In [ ]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

Отрисуйте нормальные ритмы ECG. 

In [ ]:
plt.grid()
plt.plot(np.arange(140), normal_train_data[0])
plt.title("A Normal ECG")
plt.show()

Отрисуйте ненормальные ритмы ECG.

In [ ]:
plt.grid()
plt.plot(np.arange(140), anomalous_train_data[0])
plt.title("An Anomalous ECG")
plt.show()

### Создание модели

In [ ]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])
    
    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(140, activation="sigmoid")])
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')

Обратите внимание, что автокодировщик обучается с использованием только нормальных ЭКГ, но оценивается с использованием полного набора данных.

In [ ]:
history = autoencoder.fit(normal_train_data, normal_train_data, 
          epochs=20, 
          batch_size=512,
          validation_data=(test_data, test_data),
          shuffle=True)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

Вы быстро классифицируете ЭКГ как аномальную, если ошибка реконструкции больше одного стандартного отклонения от нормальных обучающих примеров. Сначала давайте построим нормальную ЭКГ из датасета, ее реконструкцию после кодирования и декодирования и ошибку реконструкции.

In [ ]:
encoded_imgs = autoencoder.encoder(normal_test_data).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

plt.plot(normal_test_data[0],'b')
plt.plot(decoded_imgs[0],'r')
plt.fill_between(np.arange(140), decoded_imgs[0], normal_test_data[0], color='lightcoral' )
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

Создайте аналогичный график, на этот раз для аномального примера.

In [ ]:
encoded_imgs = autoencoder.encoder(anomalous_test_data).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

plt.plot(anomalous_test_data[0],'b')
plt.plot(decoded_imgs[0],'r')
plt.fill_between(np.arange(140), decoded_imgs[0], anomalous_test_data[0], color='lightcoral' )
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

### Определение аномалий

Вы можете обнаружить аномалии, вычисляя, превышают ли потери при реконструкции фиксированный порог. В этом руководстве вы рассчитаете среднюю среднюю ошибку для нормальных примеров из обучающего набора, а затем классифицируете будущие примеры как аномальные, если ошибка реконструкции превышает одно стандартное отклонение от обучающего набора.

Постройте график ошибки реконструкции на нормальных ЭКГ из обучающей выборки

In [ ]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.hist(train_loss, bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

Выберите пороговое значение, которое на одно стандартное отклонение выше среднего.

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Примечание. Существуют и другие стратегии, которые вы можете использовать для выбора порогового значения для определения аномалий. Правильный подход будет зависеть от вашего набора данных. Вы можете узнать больше по ссылкам в конце этого руководства.

Если вы исследуете ошибку реконструкции для аномальных примеров, вы заметите, что большинство из них имеют бОльшую ошибку реконструкции, чем пороговое значение. Изменяя порог, вы можете настроить [precision](https://developers.google.com/machine-learning/glossary#precision) и [recall] (https://developers.google.com/machine-learning/glossary#recall) вашего классификатора.

In [ ]:
reconstructions = autoencoder.predict(anomalous_test_data)
test_loss = tf.keras.losses.mae(reconstructions, anomalous_test_data)

plt.hist(test_loss, bins=50)
plt.xlabel("Test loss")
plt.ylabel("No of examples")
plt.show()

Классифицируйте ЭКГ как аномалию, если ошибка восстановления превышает пороговое значение.

In [ ]:
def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, preds)))
  print("Precision = {}".format(precision_score(labels, preds)))
  print("Recall = {}".format(recall_score(labels, preds)))

In [ ]:
preds = predict(autoencoder, test_data, threshold)
print_stats(preds, test_labels)

## Следующие шаги

Чтобы узнать больше об обнаружении аномалий с помощью автоэнкодеров, ознакомьтесь с этим прекрасным [интерактивным примером](https://anomagram.fastforwardlabs.com/#/), созданным Victor Dibia с помощью TensorFlow.js. 
Для практического использования вы можете узнать, как [Airbus обнаруживает аномалии в данных телеметрии МКС](https://blog.tensorflow.org/2020/04/how-airbus-detects-anomalies-iss-telemetry-data- tfx.html) с помощью TensorFlow. 
Чтобы узнать больше об основах, прочтите это [сообщение в блоге](https://blog.keras.io/building-autoencoders-in-keras.html) François Chollet. 
Для получения дополнительной информации ознакомьтесь с главой 14 из [Deep Learning](https://www.deeplearningbook.org/) Ian Goodfellow, Yoshua Bengio и Aaron Courville.